In [17]:
import requests
import pymysql
from io import StringIO
import csv

In [19]:
file_request = requests.get("https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv")
file_request.encoding = "utf-8"

file_request = file_request.text
document = csv.reader(StringIO(file_request))

In [ ]:
locales = [(linea[3], linea[4], linea[13], linea[14], linea[15][7:]) for linea in document]

In [ ]:
diccionario = {}
for i in locales[0]:
    diccionario[i] = []

In [ ]:
diccionario.keys()

dict_keys(['nombre', 'categoria', 'direccion_completa', 'barrio', ''])

In [ ]:
diccionario["comuna"] = diccionario.pop("")
diccionario.keys()

dict_keys(['nombre', 'categoria', 'direccion_completa', 'barrio', 'comuna'])

In [ ]:
claves = list(diccionario.keys())
for linea in locales[1:]:
    for i in range(len(claves)):
        diccionario[claves[i]].append(linea[i])

Just to visualize the data, I created a dataframe

In [ ]:
import pandas as pd

df = pd.DataFrame(diccionario)
df.head(10)

,nombre,categoria,direccion_completa,barrio,comuna
0,GUANTANAMERA,RESTAURANTE,"MOREAU DE JUSTO, ALICIA AV. 2092",Puerto Madero,1
1,1843 VINACOTECA,VINERIA,HUMBERTO 1º 1843,San Cristobal,3
2,2080 EXPRESS,RESTAURANTE,JURAMENTO AV. 2080,Belgrano,13
3,22,CAFE,MURGUIONDO 4188,Villa Lugano,8
4,4 EN 2 I,RESTAURANTE,"BORGES, JORGE LUIS 2095",Palermo,14
5,4 X 4 CAFE,BAR,DEL LIBERTADOR AV. 3838,Palermo,14
6,5TA AVENIDA,RESTAURANTE,SARMIENTO 814,San Nicolas,1
7,5TA AVENIDA I,RESTAURANTE,SARMIENTO 381,San Nicolas,1
8,5TA ZONA,,MURILLO 1049,Villa Crespo,15
9,650,RESTAURANTE,SAN MARTIN 650,San Nicolas,1


Obtengo mi contraseña de MySQL que por seguridad la tengo en un archivo ".txt"

I get my MySQL password from a text file for my security

In [ ]:
with open("../../password_db.txt") as contra:
    password = contra.read()

Here, I create the conection with de MySQL database engine and after that I create an *cursor* object to be able to interact with the database.

In [20]:
conection = pymysql.connect(
    host='localhost',
    user='root',
    password=password
)

cursor = conection.cursor()

cursor.execute("create database if not exists oferta_gastronomica;")
cursor.execute("use oferta_gastronomica;")

0

In [ ]:
df.columns

Index(['nombre', 'categoria', 'direccion_completa', 'barrio', 'comuna'], dtype='object')

In [ ]:
### Creating the table
query_db = """create table if not exists locales(
IdLocal int not null auto_increment,
Nombre varchar(80),
Categoria varchar(80),
Direccion varchar(80),
Barrio varchar(80),
Comuna varchar(80),
primary key (IdLocal)
)"""

In [ ]:
# Executing the query above
cursor.execute(query_db)

0

In [25]:
cursor.execute("truncate table locales;")

0

Inserting the data from the csv

In [26]:
query = """
insert into locales (nombre,categoria,direccion,barrio,comuna)
values (%s,%s,%s,%s,%s)
"""
cursor.executemany(query,locales[1:])

2823

Applying the querys

In [27]:
conection.commit()

a. Which neighbordhood has the most Pubs amounts?

In [69]:
query = """
SELECT  
barrio,
count(*) as Total
FROM locales WHERE categoria = 'PUB'
group by barrio 
order by Total desc
limit 1;
"""
cursor.execute(query)
result = cursor.fetchall()
result[0]

b. Get the number of stores by category

In [70]:
query = """
select
categoria,
count(*) as Cantidad_Total
from locales
group by categoria
order by Cantidad_Total desc;
"""
cursor.execute(query)
cursor.fetchall()

(('RESTAURANTE', 1990),
 ('CAFE', 334),
 ('BAR', 247),
 ('', 116),
 ('CONFITERIA', 55),
 ('PUB', 53),
 ('VINERIA', 22),
 ('SANDWICHERIA', 4),
 ('DELIVERY & TAKE AWAY', 2))

c. Get the number of restaurants by commune

In [75]:
query = """
select
concat("Comuna ",comuna),
count(*) as Total_Restaurantes
from locales
group by comuna
order by Total_Restaurantes desc;
"""
cursor.execute(query)
cursor.fetchall()

(('Comuna 1', 870),
 ('Comuna 14', 467),
 ('Comuna 2', 327),
 ('Comuna 3', 224),
 ('Comuna 4', 142),
 ('Comuna 13', 140),
 ('Comuna 6', 125),
 ('Comuna 7', 100),
 ('Comuna 15', 87),
 ('Comuna 5', 77),
 ('Comuna 12', 70),
 ('Comuna 9', 68),
 ('Comuna 10', 58),
 ('Comuna 11', 50),
 ('Comuna 8', 18))